<a href="https://colab.research.google.com/github/Hamoda-dabbit/Mining---classification-in-Arabic-Article/blob/main/SVC/SVC_TFIDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 1- إستيراد المكتبات

In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.isri import ISRIStemmer
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from joblib import dump

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


--------

2- استيراد ملفات البيانات

In [2]:
# df = pd.read_excel("drive/MyDrive/mining/1.xlsx")
# df = pd.read_excel("drive/MyDrive/mining/2.xlsx")
# df = pd.read_excel("drive/MyDrive/mining/3.xlsx")
df = pd.read_excel("drive/MyDrive/mining/4.xlsx")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68690 entries, 0 to 68689
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    68312 non-null  object
 1   targe   68690 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.0+ MB


------
3- shuffle  
بعثرة البيانات

In [3]:
df=shuffle(df)
df

,text,targe
24777,أخبارنا المغربية سناء الوردي بعد حرب الأرقام و...,2
30399,أكد المحامي المكلف بملف اللاعب الدولي السابق ع...,4
30133,أخبارنا المغربية عـادل الوزانـي رغم اطلاق مجمو...,3
65234,أخبارنا المغربية نوفل العمراني اعتدى لاعبو الف...,4
24900,NaN,2
...,...,...
46860,عضو اللجنة المركزية لحزب الاستقلال طالب بتخصيص...,3
42128,شرعت شركة سوني للإلكترونيات في تسويق كاميراتها...,2
15341,في ما يلي نتائج مباريات المرحلة الرابعة من الد...,4
34646,أخبارنا المغربية ــ متابعة يعتزم أعضاء من جمعي...,0


------
4- clean
تنظيف البيانات

In [4]:
print(np.sum(df.isnull().any(axis=1)))

clean_df= df.dropna()
print(np.sum(clean_df.isnull().any(axis=1)))
#-------------
print (df.notnull().any(axis = 0))
print(len(df))
#-------------
def remove_hashtag(df, col = 'text'):
    for letter in r'#.][!XR':
      df[col] = df[col].astype(str).str.replace(letter,'', regex=True)
        
remove_hashtag(clean_df)
clean_df
#-------------
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
#-------------
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text
#-------------    
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)
#-------------
def processDocument(doc, stemmer): 

    #Replace @username with empty string
    doc = re.sub(r'@[^\s]+', ' ', doc)
    doc = re.sub(r'_', ' ', doc)
    doc = re.sub(r'\n', ' ', doc)
    doc = re.sub(r'[a-z,A-Z]', '', doc)
    doc = re.sub(r'\d', '', doc)
    #Convert www.* or https?://* to " "
    doc = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',doc)
    #Replace #word with word
    doc = re.sub(r'#([^\s]+)', r'\1', doc)
    # remove punctuations
    doc= remove_punctuations(doc)
    # normalize the tweet
    doc= normalize_arabic(doc)
    # remove repeated letters
    doc=remove_repeating_char(doc)
    #stemming
    doc = stemmer.stem(doc)
    
    return doc

stemmer = ISRIStemmer()
clean_df["text"] = clean_df['text'].apply(lambda x: processDocument(x, stemmer))
print(clean_df)

378
0
text     True
targe    True
dtype: bool
68690
                                                    text  targe
24777  اخبارنا المغربيه سناء الوردي بعد حرب الارقام و...      2
30399  اكد المحامي المكلف بملف الاعب الدولي السابق عب...      4
30133  اخبارنا المغربيه عادل الوزاني رغم اطلاق مجموعه...      3
65234  اخبارنا المغربيه نوفل العمراني اعتدي لاعبو الف...      4
5562   تمكن امن العرائش اليوم الخميس من احباط محاوله ...      1
...                                                  ...    ...
46860  عضو الجنه المركزيه لحزب الاستقلال طالب بتخصيص ...      3
42128  شرعت شركه سوني لالكترونيات في تسويق كاميراتها ...      2
15341  في ما يلي نتائج مباريات المرحله الرابعه من الد...      4
34646  اخبارنا المغربيه متابعه يعتزم اعضاء من جمعيه م...      0
33767  نشرت الفنانه البنانيه نوال الزغبي صورا علي حسا...      0

[68312 rows x 2 columns]


----

5- tokenizer
تقسيم كل نص على مستوى الكلمة

In [5]:
tokenizer = RegexpTokenizer(r'\w+')
clean_df["text"] = clean_df["text"].apply(tokenizer.tokenize)
print(clean_df)

                                                    text  targe
24777  [اخبارنا, المغربيه, سناء, الوردي, بعد, حرب, ال...      2
30399  [اكد, المحامي, المكلف, بملف, الاعب, الدولي, ال...      4
30133  [اخبارنا, المغربيه, عادل, الوزاني, رغم, اطلاق,...      3
65234  [اخبارنا, المغربيه, نوفل, العمراني, اعتدي, لاع...      4
5562   [تمكن, امن, العرائش, اليوم, الخميس, من, احباط,...      1
...                                                  ...    ...
46860  [عضو, الجنه, المركزيه, لحزب, الاستقلال, طالب, ...      3
42128  [شرعت, شركه, سوني, لالكترونيات, في, تسويق, كام...      2
15341  [في, ما, يلي, نتائج, مباريات, المرحله, الرابعه...      4
34646  [اخبارنا, المغربيه, متابعه, يعتزم, اعضاء, من, ...      0
33767  [نشرت, الفنانه, البنانيه, نوال, الزغبي, صورا, ...      0

[68312 rows x 2 columns]


---
6- Delete StopWords

حذف الكلمات غير المؤثرة

In [6]:
stopwords_list = stopwords.words('arabic')
listToStr = ' '.join([str(elem) for elem in stopwords_list]) 
print(listToStr)
#-------------
clean_df["text"] = clean_df["text"].apply(lambda x: [item for item in x if item not in stopwords_list])
((clean_df))

إذ إذا إذما إذن أف أقل أكثر ألا إلا التي الذي الذين اللاتي اللائي اللتان اللتيا اللتين اللذان اللذين اللواتي إلى إليك إليكم إليكما إليكن أم أما أما إما أن إن إنا أنا أنت أنتم أنتما أنتن إنما إنه أنى أنى آه آها أو أولاء أولئك أوه آي أي أيها إي أين أين أينما إيه بخ بس بعد بعض بك بكم بكم بكما بكن بل بلى بما بماذا بمن بنا به بها بهم بهما بهن بي بين بيد تلك تلكم تلكما ته تي تين تينك ثم ثمة حاشا حبذا حتى حيث حيثما حين خلا دون ذا ذات ذاك ذان ذانك ذلك ذلكم ذلكما ذلكن ذه ذو ذوا ذواتا ذواتي ذي ذين ذينك ريث سوف سوى شتان عدا عسى عل على عليك عليه عما عن عند غير فإذا فإن فلا فمن في فيم فيما فيه فيها قد كأن كأنما كأي كأين كذا كذلك كل كلا كلاهما كلتا كلما كليكما كليهما كم كم كما كي كيت كيف كيفما لا لاسيما لدى لست لستم لستما لستن لسن لسنا لعل لك لكم لكما لكن لكنما لكي لكيلا لم لما لن لنا له لها لهم لهما لهن لو لولا لوما لي لئن ليت ليس ليسا ليست ليستا ليسوا ما ماذا متى مذ مع مما ممن من منه منها منذ مه مهما نحن نحو نعم ها هاتان هاته هاتي هاتين هاك هاهنا هذا هذان هذه هذي هذين هكذا هل هلا هم هما هن هنا هنا

,text,targe
24777,"[اخبارنا, المغربيه, سناء, الوردي, حرب, الارقام...",2
30399,"[اكد, المحامي, المكلف, بملف, الاعب, الدولي, ال...",4
30133,"[اخبارنا, المغربيه, عادل, الوزاني, رغم, اطلاق,...",3
65234,"[اخبارنا, المغربيه, نوفل, العمراني, اعتدي, لاع...",4
5562,"[تمكن, امن, العرائش, اليوم, الخميس, احباط, محا...",1
...,...,...
46860,"[عضو, الجنه, المركزيه, لحزب, الاستقلال, طالب, ...",3
42128,"[شرعت, شركه, سوني, لالكترونيات, تسويق, كاميرات...",2
15341,"[يلي, نتائج, مباريات, المرحله, الرابعه, الدوري...",4
34646,"[اخبارنا, المغربيه, متابعه, يعتزم, اعضاء, جمعي...",0


----
7- عمل تابع لحساب الدقة

In [ ]:
def calculate_results(y_true, y_pred):
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

------------
8- classifier:SVC & TfIdfVectorizer

with Cross validation



التنصنيف عبر خوارزمية

SVC

مع ميزة

TfIdfVectorizer

واستخدام خوارزمية

Cross validation

لتوزيع عينات الاختبار والتأكد من عدم وجود

Overfitting

In [8]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    ngram_range=(1, 1),
    max_features =10000)

model = Pipeline([
                    ("tfidf", word_vectorizer), # convert words to numbers using tfidf
                    ("clf", SVC()) # model the text
])

from sklearn.model_selection import KFold
kf = KFold(n_splits=4,shuffle=False)
result_Pipeline=[]
# for i in range(0,len(clean_df)):
y1 = clean_df['targe']
X1 = clean_df['text']

kf.split(X1)
j =0
for train_index, test_index in kf.split(X1):
#     # Split train-test
    X_train, X_test = X1.iloc[train_index], X1.iloc[test_index]
    y_train, y_test = y1.iloc[train_index], y1.iloc[test_index]
    model.fit(X_train.astype('str'), y_train)
    dump(model, "svm_model.joblib")
    
    y_pred=(model.predict(X_test.astype('str')))
    result_Pipeline.append(calculate_results(y_test, y_pred))

result_Pipeline

[{'accuracy': 95.05796931725027,
  'f1': 0.9505694001631746,
  'precision': 0.9506118505028125,
  'recall': 0.9505796931725027},
 {'accuracy': 94.81203888043096,
  'f1': 0.9480809275456434,
  'precision': 0.9480823765319017,
  'recall': 0.9481203888043096},
 {'accuracy': 95.36831010656985,
  'f1': 0.9536585368304017,
  'precision': 0.9536470537086641,
  'recall': 0.9536831010656985},
 {'accuracy': 95.2277784283874,
  'f1': 0.9522974669838242,
  'precision': 0.9523328416109652,
  'recall': 0.952277784283874}]

In [ ]:
accuracy=0
f1=0
precision=0
recall=0
for i in result_Pipeline:
  accuracy+=(i['accuracy'])
  f1+=(i['f1'])
  precision+=(i['precision'])
  recall+=(i['recall'])

accuracy=accuracy/len(result_Pipeline)
f1=f1/len(result_Pipeline)
precision=precision/len(result_Pipeline)
recall=recall/len(result_Pipeline)
# (result_DecisionTreeClassifier)
results_Pipeline=[{'accuracy':accuracy,'f1':f1,'precision':precision,'recall':recall}]
results_Pipeline

[{'accuracy': 95.11652418315961,
  'f1': 0.951151582880761,
  'precision': 0.9511685305885859,
  'recall': 0.9511652418315961}]

-------
--------
-------

9-Results

النتائج

نتائج التدريب لكل قاعدة بيانات

In [18]:
print('Pipeline results in dataset 1:')
results_Pipeline

Pipeline results in dataset 1:


[{'accuracy': 96.89533114224355,
  'f1': 0.9689985341982958,
  'precision': 0.9690937439771347,
  'recall': 0.9689533114224355}]

In [10]:
print('Pipeline results in dataset 2:')
results_Pipeline

Pipeline results in dataset 2:


[{'accuracy': 96.70650040411039,
  'f1': 0.9670976908703979,
  'precision': 0.9671582117740604,
  'recall': 0.9670650040411037}]

In [10]:
print('Pipeline results in dataset 3:')
results_Pipeline

Pipeline results in dataset 3:


[{'accuracy': 95.45393519602939,
  'f1': 0.9545394323284653,
  'precision': 0.9545786090267925,
  'recall': 0.9545393519602939}]

In [10]:
print('Pipeline results in dataset 4:')
results_Pipeline

Pipeline results in dataset 4:


[{'accuracy': 95.11652418315961,
  'f1': 0.951151582880761,
  'precision': 0.9511685305885859,
  'recall': 0.9511652418315961}]

In [12]:
Final_Result=[
  {'accuracy': 96.89533114224355,
  'f1': 0.9689985341982958,
  'precision': 0.9690937439771347,
  'recall': 0.9689533114224355}
  ,
 {'accuracy': 96.70650040411039,
  'f1': 0.9670976908703979,
  'precision': 0.9671582117740604,
  'recall': 0.9670650040411037}
  ,
 {'accuracy': 95.45393519602939,
  'f1': 0.9545394323284653,
  'precision': 0.9545786090267925,
  'recall': 0.9545393519602939}
  ,
 {'accuracy': 95.11652418315961,
  'f1': 0.951151582880761,
  'precision': 0.9511685305885859,
  'recall': 0.9511652418315961}
  ]

accuracy=0
f1=0
precision=0
recall=0
for i in Final_Result:
  accuracy+=(i['accuracy'])
  f1+=(i['f1'])
  precision+=(i['precision'])
  recall+=(i['recall'])

accuracy=accuracy/len(Final_Result)
f1=f1/len(Final_Result)
precision=precision/len(Final_Result)
recall=recall/len(Final_Result)
# (result_DecisionTreeClassifier)
Final_Result=[{'accuracy':accuracy,'f1':f1,'precision':precision,'recall':recall}]

print('Final_Result:')
(Final_Result)

Final_Result:


[{'accuracy': 96.04307273138573,
  'f1': 0.96044681006948,
  'precision': 0.9604997738416433,
  'recall': 0.9604307273138573}]